# 모델링 - 토큰 전환

2021.02.24 (수) 김채형

In [1]:
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
def PreprocessingHashtag(path):
    
    import pandas as pd
    import re
    
    # 데이터 불러오기
    data = pd.read_csv(path, index_col=0)
    
    # 해시태그에서 특수문자 제거 후 우물 정(#) 기준으로 분리
    p = re.compile(r'[가-힣# ]+')
    data['hashtags_splitted'] = data['hashtags'].apply(lambda x: ''.join(p.findall(str(x))).split('#'))
    
    # 빈 해시태그 제거
    data['hashtags_completed'] = ''
    for i in range(len(data)):
        ls = [word for word in data.loc[i, 'hashtags_splitted'] if word!='']
        data.loc[i, 'hashtags_completed'] = ls
        
    # 장소를 해시태그에 추가
    l = data['location'].fillna('')
    for i in range(len(data)):
        result = re.compile(r'[가-힣]+').search(l[i])
        if result!=None:
            data.loc[i, 'hashtags_completed'].append(result.group())

    # 컬럼 삭제
    data.drop(['hashtags', 'hashtags_splitted'], axis=1, inplace=True)
    
    # 컬럼명 변경
    data.rename({'hashtags_completed':'hashtags'}, axis=1, inplace=True)
    
    return data

In [3]:
df = PreprocessingHashtag('./data/dataset.csv')
df

image_name                                          image_url  \
0         일상_0.jpg  https://scontent-ssn1-1.cdninstagram.com/v/t51...   
1         일상_1.jpg  https://scontent-ssn1-1.cdninstagram.com/v/t51...   
2         일상_2.jpg  https://scontent-ssn1-1.cdninstagram.com/v/t51...   
3         일상_3.jpg  https://scontent-ssn1-1.cdninstagram.com/v/t51...   
4         일상_4.jpg  https://scontent-ssn1-1.cdninstagram.com/v/t51...   
...            ...                                                ...   
5386  펫스타그램_95.jpg  https://scontent-ssn1-1.cdninstagram.com/v/t51...   
5387  펫스타그램_96.jpg  https://scontent-ssn1-1.cdninstagram.com/v/t51...   
5388  펫스타그램_97.jpg  https://scontent-ssn1-1.cdninstagram.com/v/t51...   
5389  펫스타그램_98.jpg  https://scontent-ssn1-1.cdninstagram.com/v/t51...   
5390  펫스타그램_99.jpg  https://scontent-ssn1-1.cdninstagram.com/v/t51...   

                                                content              location  \
0                                         날씨 너무 죠타🌸🌼🌸🌼🍃  Gwangju, South Korea   
1     날씨 좋은 월요일 맛점💕\n\n⠀\n⠀\n⠀\n⠀\n⠀\n⠀\n⠀\n⠀\n⠀\n#월...            슈엘뷰티샵&아카데미   
2        곧 5️⃣년 된다는 사실에 놀라고 졸업👩🏻‍🎓이라는 사실에 한번 더 놀라고 .. 🤭           Naive Place   
3                              나 뭐가 달라진 거 같아~~~~??????😁                Jeonju   
4     ♥\n\n요즘 공기 봄 냄새나는거같아서 좋다☺️\n쌩얼인데 어쩌라고\n\n#광주속눈...                   매곡동   
...                                                 ...                   ...   
5386                                      달이,, 준며들다,,__                   NaN   
5387  왜..그렇게 그윽하게 보는거야..?\n.\n.\n.\n#코리안쇼콧 #kitty #고...                   NaN   
5388                                             Rookie                   NaN   
5389                      나도 육포를 달라개!\n자기들 끼리 육포 먹고..__                   NaN   
5390                        사람처럼 팔 배고 낸내 __ #전요미하고싶은거다해                   NaN   

                                               hashtags  
0     [광주, 일상, 일상스타그램, 데일리룩, 데일리, 소통, 소통해요, 맞팔, 선팔, ...  
1     [월요일, 출근, 뷰티샵, 대구토탈뷰티샵, 대구속눈썹, 대구왁싱, 슈엘뷰티, 사진맛...  
2     [광주, 동명동, 양림동, 충장로, 조대, 조대후문, 수완지구, 상무지구, 서울, ...  
3     [맞팔, 팔로우, 팔로워, 셀카, 인스타, 셀피, 일상, 데일리, 좋아요반사, 좋아...  
4     [광주속눈썹연장, 속눈썹연장잘하는곳, 뷰티어스, 뷰티어스래쉬, 속눈썹연장, 일상, ...  
...                                                 ...  
5386  [개스타그램, 멍스타그램, 반려견, 반려동물, 강아지, 일상, 댕댕이, 독스타그램,...  
5387  [코리안쇼콧, 고양이, 반려묘, 일상, 미묘, 소통, 좋아요, 데일리, 펫스타그램,...  
5388             [크레스티드게코, 릴리화이트, 파충류, 도마뱀, 데일리, 펫스타그램]  
5389  [애견, 강아지스타그램, 멍스타그램, 멍멍스타그램, 푸들, 푸들믹스견, 개, 좋아요...  
5390  [전요미하고싶은거다해, 요미, 부산, 멍스타그램, 견스타그램, 강아지, 반려견, 반...  

[5391 rows x 5 columns]

In [4]:
hashtags_list = [l for l in df['hashtags'].values]

In [5]:
from konlpy.tag import Mecab

tokenizer = Mecab()
tokenized = []
for hashtags in hashtags_list:
    temp = tokenizer.morphs(' '.join(hashtags))
    tokenized.append(temp)

In [6]:
from gensim.models import Word2Vec

model = Word2Vec(sentences = tokenized, 
                 size = 100, 
                 window = 10, 
                 min_count = 5, 
                 workers = 4, 
                 sg = 1)

In [7]:
model.wv.vectors.shape

(2685, 100)

In [8]:
model.wv.most_similar('신촌')

[('역삼', 0.9733798503875732),
 ('압구정', 0.952819287776947),
 ('힙합', 0.9523053765296936),
 ('삼덕동', 0.9517554640769958),
 ('남포', 0.9489603042602539),
 ('가로수길', 0.9309589266777039),
 ('불로동', 0.9280554056167603),
 ('건대', 0.9248971343040466),
 ('마린', 0.9218472838401794),
 ('신사', 0.9172803163528442)]

In [9]:
def NTokenConversion(model, t_list, alpha, n):

    res = []
    for t in t_list:
        similar_tokens = model.wv.most_similar(t[0])
        for i in range(len(similar_tokens)):
            if (alpha**n)*(similar_tokens[i][1])>=0.7:
                res.append(similar_tokens[i])

    return res

In [10]:
def TokenConversion(model, common_tokens, alpha):
    
    # 토큰 전환
    ls1 = NTokenConversion(model, common_tokens, alpha, n=0)
    ls2 = NTokenConversion(model, ls1, alpha, n=1)
    ls3 = NTokenConversion(model, ls2, alpha, n=2)
    ls4 = NTokenConversion(model, ls3, alpha, n=3)
    ls5 = NTokenConversion(model, ls4, alpha, n=4)
    ls = ls1 + ls2 + ls3 + ls4 + ls5

    # 중복 토큰 제거
    temp = []
    result = []
    for i in range(len(ls)):
        if ls[i][0] not in temp:
            temp.append(ls[i][0])
            result.append(ls[i])

    return result

In [11]:
# 토큰 전환
common_tokens = [('신촌', 0.91353440284729), ('가로수길', 0.9186334013938904), ('홍대', 0.7084668278694153)]
tokens_converted = TokenConversion(model, common_tokens, alpha=0.7)
tokens_converted

[('역삼', 0.9733798503875732),
 ('압구정', 0.952819287776947),
 ('힙합', 0.9523053765296936),
 ('삼덕동', 0.9517554640769958),
 ('남포', 0.9489603042602539),
 ('가로수길', 0.9309589266777039),
 ('불로동', 0.9280554056167603),
 ('건대', 0.9248971343040466),
 ('마린', 0.9218472838401794),
 ('신사', 0.9172803163528442),
 ('신촌', 0.9309588670730591),
 ('동성로', 0.9150162935256958),
 ('한남동', 0.9066246151924133),
 ('신사동', 0.9047465324401855),
 ('논현', 0.8896282911300659),
 ('미용인', 0.8854705691337585),
 ('이태원', 0.9145087599754333),
 ('강남', 0.881061315536499),
 ('연남', 0.836570143699646),
 ('합정', 0.8206409215927124)]